In [ ]:
import requests
from requests import Response
aqi_url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=JSON'

res: Response = requests.get(aqi_url)
if res.status_code == 200:
    print('下載成功')
else:
    print('下載失敗')

#自訂的欄位驗証
from pydantic import BaseModel, Field, field_validator

class Site(BaseModel):
    site_name: str = Field(alias='sitename')
    county: str
    aqi: int
    status: str
    pm25: float = Field(alias='pm2.5')
    so2: float
    co: float

    @field_validator('pm25', 'so2', 'co', mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        if value == '':
            return '0.0'
        else:
            return value

class AQI(BaseModel):
    records: list[Site]

data: AQI = AQI.model_validate_json(res.text)


In [ ]:
all_sites:list[dict] = data.model_dump()['records']
all_sites #python的基本資料結構

In [8]:
import pandas as pd
df1 = pd.DataFrame(all_sites)
df1


,site_name,county,aqi,status,pm25,so2,co
0,屏東(枋山),屏東縣,41,良好,14.0,0.0,0.16
1,大甲（日南國小）,臺中市,24,良好,3.0,0.2,0.25
2,新北(樹林),新北市,25,良好,8.0,0.2,0.26
3,屏東（琉球）,屏東縣,21,良好,4.0,2.4,0.13
4,臺南（麻豆）,臺南市,23,良好,8.0,0.2,0.18
...,...,...,...,...,...,...,...
995,頭份,苗栗縣,19,良好,7.0,1.1,0.17
996,新竹,新竹市,18,良好,6.0,0.7,0.21
997,竹東,新竹縣,13,良好,2.0,0.0,0.11
998,湖口,新竹縣,22,良好,8.0,0.3,0.13


In [7]:
df1[df1['aqi'] <= 13]

,site_name,county,aqi,status,pm25,so2,co
956,宜蘭,宜蘭縣,13,良好,4.0,0.4,0.16
997,竹東,新竹縣,13,良好,2.0,0.0,0.11
